# Types of Databases



### Relational
- Represent and store data in tables and rows.
- Use Structured Query Language (`SQL`), which is like a programming language for relational databases.
- Examples include `SQLite`, `MySQL`, `PostreSQL`, `T-SQL`.

### Non-relational
- Represent and store data as collections of documents, objects, key-value stores, or heirarchal data formats.
- Also known as "NoSQL" databases.
- Examples include `JSON`, `MongoDB`.

# SQL: Structured Query Language

- Programming language for storing, manipulating, and retreiving data stored in a relational database.
- All relational database management systems (e.g. `MySQL`, `SQLite`, `PostgreSQL`) use `SQL` as their standard database landuage.
- `SQL` can have different *dialects* that contain small, subtle differences (just like there are different *accents* in the english language). For example, a string-like data type in `MySQL` is defined as **`STRING`**, while in `SQLite` it is defined as **`TEXT`**. 
- However, the vast majority of `SQL` is the same in all relational database management systems.

# What will we learn?

In this lesson, we will learn how to retreive astronomical data from a `SQLite` database.

- How to retrieve data from a database
- How to select a subset of the data
- How to sort a query's results
- How to combine data from multiple tables

We will first learn how to do this through the `SQLite` command line interface, then we will learn how to accomplish the same tasks using Python.

# The `hipparcos.db` Database

In this lesson, we will use with the `hipparcos.db` database, which stores data from the Extended Hipparcos Compilation (XHIP), created
by Erik Anderson and Charles Francis.

It contains:
- Spectral classificaitons
- Radial velocities
- Iron abundances
- Stellar classifications
- Stellar ages
- 2MASS photometry

The data can be downloaded from [this page](http://cdsarc.u-strasbg.fr/viz-bin/cat/V/137D):

In [1]:
import os

filename = 'main.fits.gz'
if not os.path.exists(filename):
    !wget http://cdsarc.u-strasbg.fr/viz-bin/nph-Cat/fits.gz?V/137D/main.dat.gz -O main.fits.gz

In [2]:
filename = 'photo.fits.gz'

if not os.path.exists(filename):
    !wget http://cdsarc.u-strasbg.fr/viz-bin/nph-Cat/fits.gz?V/137D/photo.dat.gz -O photo.fits.gz

More information about the Extended Hipparcos Compilation:

- https://arxiv.org/abs/1108.4971
- http://cdsarc.u-strasbg.fr/viz-bin/cat/V/137D

The `main.fits` file stores Astrometry, spectrography, space motions, and exoplanet indications.  The `photo.fits` file stores photometry information.

## Reading main

We can use Astropy to open the file and read info.

In [3]:
from astropy.io import fits

hdul = fits.open('main.fits.gz')
hdul.info()

Filename: main.fits.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     112   ()      
  1  main.dat      1 TableHDU       372   117955R x 55C   [I6, A6, A41, I1, F12.8, F12.8, F6.2, F8.2, F8.2, F6.2, F6.2, F5.2, F6.2, F5.2, I1, I1, F12.8, F12.8, F7.2, F4.1, F8.2, F8.2, F6.1, F7.1, F6.1, I4, F5.1, A26, I3, I1, F7.2, F6.2, A1, F5.2, F4.2, A1, F4.1, F4.1, F4.1, F6.1, F6.1, F6.1, F5.1, F7.2, I5, F6.4, F6.1, I7, I5, I7, I5, I4, I7, I1, A5]   


And read the data into a table.

It looks like there are some issue in the file, so we have to run `verify` with the `fix` argument.

In [4]:
from astropy.table import Table

with fits.open('main.fits.gz') as data:
    data.verify('fix')
    table = Table.read(data, format='fits')

 [astropy.io.fits.verify]


In [5]:
table.info

<Table length=117955>
  name    dtype    unit    format 
-------- ------- -------- --------
     HIP   int32             {:6d}
    Comp    str6                  
 Classes   str41                  
      Gr   int32             {:1d}
   RAdeg float64      deg {:12.8f}
   DEdeg float64      deg {:12.8f}
     Plx float64      mas  {:6.2f}
    pmRA float64 mas / yr  {:8.2f}
    pmDE float64 mas / yr  {:8.2f}
 e_RAdeg float64      mas  {:6.2f}
 e_DEdeg float64      mas  {:6.2f}
   e_Plx float64      mas  {:5.2f}
  e_pmRA float64 mas / yr  {:6.2f}
  e_pmDE float64 mas / yr  {:5.2f}
   r_HIP   int32             {:1d}
    r_pm   int32             {:1d}
    GLon float64      deg {:12.8f}
    GLat float64      deg {:12.8f}
    Dist float64       pc  {:7.2f}
  e_Dist float64        %  {:4.1f}
  pmGLon float64 mas / yr  {:8.2f}
  pmGLat float64 mas / yr  {:8.2f}
       X float64       pc  {:6.1f}
       Y float64       pc  {:7.1f}
       Z float64       pc  {:6.1f}
    RGal   int32       pc    {:4d

We can also read the data using Pandas, so the result goes into a `DataFrame`.

In [6]:
import pandas as pd

In [7]:
with fits.open('main.fits.gz') as data:
    data.verify('fix')
    main = pd.DataFrame(data[1].data)

 [astropy.io.fits.verify]


In [8]:
main.shape

(117955, 55)

In [9]:
main.head()

,HIP,Comp,Classes,Gr,RAdeg,DEdeg,Plx,pmRA,pmDE,e_RAdeg,...,e,phi,a,b,c,L,Rmin,Rmax,Npl,Mpl
0,1,,"*,IR",0,0.000912,1.089013,4.55,-4.58,-1.61,1.29,...,0.0000,0.0,0,0,0,0,0,0,0,
1,2,A,"*,PM*,IR",0,0.003797,-19.498837,20.85,179.70,1.40,0.95,...,0.0000,0.0,0,0,0,0,0,0,0,
2,3,AB,"**,*,Em*,IR,UV",0,0.005008,38.859286,2.26,4.28,-3.42,0.31,...,0.1114,-5.2,8504,8451,947,8398,7557,9451,0,
3,4,,"*,IR",0,0.008382,-51.893546,7.45,62.95,0.56,0.40,...,0.0000,0.0,0,0,0,0,0,0,0,
4,5,,"*,IR",0,0.009965,-40.591224,3.88,1.54,8.30,0.55,...,0.0000,0.0,0,0,0,0,0,0,0,


## Reading photometry

Same with the photometry data: we can use Astropy to read the info and the data, and Pandas to put the results into a `DataFrame`.

In [10]:
hdul = fits.open('photo.fits.gz')
hdul.info()

Filename: photo.fits.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     112   ()      
  1  photo.dat     1 TableHDU       236   117955R x 36C   [I6, F7.4, F6.4, A1, F5.2, F5.2, F6.2, A1, F6.3, F6.3, F5.2, F5.2, F6.3, F6.3, F6.3, F6.3, F5.3, F5.3, F5.3, A17, A3, F6.3, F5.2, F5.3, F4.2, F6.3, F6.3, F5.2, F5.2, F5.2, F5.2, F5.2, F5.2, F6.2, F9.2, F5.2]   


In [11]:
with fits.open('photo.fits.gz') as data:
    data.verify('fix')
    table2 = Table.read(data, format='fits')

In [12]:
table2.info

<Table length=117955>
  name   dtype  unit  format
------- ------- ---- -------
    HIP   int32   pc   {:6d}
  Hpmag float64  mag {:7.4f}
e_Hpmag float64  mag {:6.4f}
m_Hpmag    str1             
  Hpmax float64  mag {:5.2f}
  Hpmin float64  mag {:5.2f}
    Per float64    d {:6.2f}
   Hvar    str1  mag        
   Umag float64  mag {:6.3f}
   Bmag float64  mag {:6.3f}
   Vmag float64  mag {:5.2f}
   Rmag float64  mag {:5.2f}
   Imag float64  mag {:6.3f}
   Jmag float64  mag {:6.3f}
   Hmag float64  mag {:6.3f}
   Kmag float64  mag {:6.3f}
 e_Jmag float64  mag {:5.3f}
 e_Hmag float64  mag {:5.3f}
 e_Kmag float64  mag {:5.3f}
  2MASS   str17             
    q2M    str3             
    B-V float64  mag {:6.3f}
    V-I float64  mag {:5.2f}
  e_B-V float64  mag {:5.3f}
  e_V-I float64  mag {:4.2f}
  HpMag float64  mag {:6.3f}
   UMag float64  mag {:6.3f}
   BMag float64  mag {:5.2f}
   VMag float64  mag {:5.2f}
   RMag float64  mag {:5.2f}
   IMag float64  mag {:5.2f}
   JMag float64  mag 

In [13]:
with fits.open('photo.fits.gz') as data:
    data.verify('fix')
    photo = pd.DataFrame(data[1].data)

In [14]:
photo.shape

(117955, 36)

## Writing the database

Now we can replicate the database from the original version of this tutorial.

First I'll connect to the database file, creating it if it doesn't already exist.

In [15]:
import sqlite3

con = sqlite3.connect('hipparcos.db')

The main table has two columns that are the same except for capitalization, which SQL cannot handle.  So I'll rename the second column before trying to write the table.

In [16]:
main.rename(columns={'B_age': 'B_age2'}, inplace=True)

Now we can add the table to the database.

In [17]:
main.to_sql(name='data', con=con, if_exists='replace')

The photometry table has the same problem with repeated column names, but we can rename them the same way.

In [18]:
mapper = {'HpMag': 'HpMag2',
          'HMag': 'HMag2',
          'IMag': 'IMag2',
          'JMag': 'JMag2',
          'KMag': 'KMag2',
          'UMag': 'UMag2',
          'VMag': 'VMag2',
          'RMag': 'RMag2',
          'BMag': 'BMag2'}
photo.rename(columns=mapper, inplace=True)

And then add the second table to the database.

In [19]:
photo.to_sql(name='photometry', con=con, if_exists='replace')

Now the examples and exercises in the next section work.

# Opening and Navigating the Database

To connect to the database, we can use the `sqlite3` command line interface:

```
>>> sqlite3 hipparcos.db
```

```
SQLite version 3.13.0 2016-05-18 10:57:30
Enter ".help" for usage hints.
```

**Excercise 1:** Try connecting to the `hipparcos.db` database.  Use the `.help` command to determine which command to use to list the names of tables of the database and run it.

To make the output a bit more human-readable, let's run the following commands:

```
.mode column
.header on
```

In `sqlite`, we can list the columns of a table using the `PRAGMA table_info()` command:

```
sqlite> PRAGMA table_info(data);
```

**Excercise 2:** List all of the columns in the `photometry` table.  How many columns are there?

# Selecting everything from a table

```SQL
SELECT *
FROM <table_name>;
```

for example:

```SQL
SELECT *
FROM data;
```

Note that we can also limit the number of results returned:

```SQL
SELECT *
FROM data
LIMIT 10;
```

**Excercise 3:** Write a query that selects all columns from the `photometry` table and limits the results to 50 rows.

*Solution*:

```SQL
SELECT *
FROM photometry
LIMIT 50;
```

# Filtering specific columns

```SQL
SELECT <column1_name>, <column2_name>, ..., <columnN_name>
FROM <table_name>;
```

for example:

```SQL
SELECT HIP, RV, e_RV
FROM data;
```

**Exercise 4**: Write a query that selects the Hipparcos Identifier, the Hipparcos magnitude, and the variability period from the `photometry` table.  You may need to list the table's columns or refer to the documentation to check their column name.

# Filtering specific rows

```SQL
SELECT <column1_name>, <column2_name>, ..., <columnN_name>
FROM <table_name>
WHERE <column_name> <condition>;
```

for example:

```SQL
SELECT HIP, RV, e_RV
FROM data
WHERE RV != 0;
```

More conditions can be added to the query using the `AND` command.  There is no limit to the number of conditions you can put on a query.


```SQL
SELECT <column1_name>, <column2_name>, ..., <columnN_name>
FROM <table_name>
WHERE <column_name> <condition>
AND <column_name> <condition>
...
AND <column_name> <condition>;
```

for example:
```SQL
SELECT HIP, RV, e_RV
FROM data
WHERE RV != 0
AND e_RV < 1;
```

**Exercise 5:** Write a query that selects the Hipparcos indentifier, the Johnson B apparent magnitute, the Johnson V apparent magnitude, and the variability period.  Only select sources whose variability period and B-magnitute are non-zero, and have a B-magnitude less than 10.  

# Sorting results

```SQL
SELECT <column1_name>, <column2_name>, ..., <columnN_name>
FROM <table_name>
WHERE <column_name> <condition>
AND <column_name> <condition>
...
AND <column_name> <condition>
ORDER BY <column_name>;
```

for example:

```SQL
SELECT HIP, Bmag, Vmag, Per
FROM photometry
WHERE Per != 0
AND Bmag != 0
AND Bmag < 10
ORDER BY Vmag;
```

**Exercise 6:** Write a query that selects the Hipparcos identifer, the Johnson B and V apparent magnitudes, and the variability period.  Only select sources whose V-magnitude is greater than 11, and whose variability type is 'P'.  Order the results by the variability period. 

# Joining Tables

<img src="https://image.slidesharecdn.com/2-background-140403152126-phpapp02/95/database-2-background-30-638.jpg?cb=1396538564">

```SQL
SELECT <column1_name>, <column2_name>, ..., <columnN_name>
FROM <table1_name>
JOIN <table2_name> ON <condition>;
```

for example:

```SQL
SELECT data.HIP, pmRA, pmDE, Bmag, Vmag
FROM data
JOIN photometry ON data.HIP = photometry.HIP;
```

Note that `WHERE`, `AND`, and `ORDER BY` conditions can be applied to the query after the `JOIN`, for example:

```SQL
SELECT data.HIP, pmRA, pmDE, Bmag, Vmag
FROM data
JOIN photometry ON data.HIP = photometry.HIP
WHERE Vmag > 11
AND Bmag > 11
ORDER BY Vmag;
```

**Exercise 7:** Using `JOIN`, write a query that selects the Hipparcos identifier and the radial velocity from the `data` table, as well as the Johnson B and V apparent magnitudes from the `photometry` table. Only select sources whose radial velocity are greater than 0, and order the results by the Johnson B magnitude.

# Programming with Databases using Python

We have now learned how to retreive, filter, and sort data from a `SQL` database using the `SQLite` command line interface.  The same tasks can be performed in Python, using the `sqlite3` library.

In [20]:
import sqlite3

To connect to the database, we can use the `sqlite3` `connect()` method:

In [21]:
connection = sqlite3.connect("hipparcos.db")

Next, we set up a `cursor` object, which will later allow us to execute `SQL` commands:

In [22]:
cursor = connection.cursor()

Executing a database query is not as simple as passing the hard-coded query to the `cursor.execute()` command:

```python
cursor.execute("<SQL COMMAND>;")
```

In [23]:
sql_command = """
    SELECT HIP, RV, e_RV
    FROM data
    WHERE RV != 0
    AND e_RV < 1
    LIMIT 10;
    """
cursor.execute(sql_command)

To retreive the results of the query, we can use the `cursor.fetchall()` method.  This returns a list of tuples, with each element in the list being a row of results:


In [24]:
results = cursor.fetchall()
for result in results:
    print(result)

(14, 21.61, 0.27)
(19, 6.3, 0.7)
(20, 37.8, 0.4)
(21, -11.72, 0.16)
(25, 3.0, 0.7)
(31, 3.52, 0.21)
(33, -18.2, 0.5)
(38, 10.0, 0.4)
(39, -1.9, 0.2)
(43, -33.3, 0.4)


We can also use Pandas to run the query and put the results into a `DataFrame`:

In [25]:
df = pd.read_sql_query(sql_command, con)
df

,HIP,RV,e_RV
0,14,21.61,0.27
1,19,6.30,0.70
2,20,37.80,0.40
3,21,-11.72,0.16
4,25,3.00,0.70
5,31,3.52,0.21
6,33,-18.20,0.50
7,38,10.00,0.40
8,39,-1.90,0.20
9,43,-33.30,0.40


Here's a more complex example involving a join.

In [26]:
import sqlite3

connection = sqlite3.connect("hipparcos.db")
cursor = connection.cursor()

sql_command = """
    SELECT data.HIP, RV, Bmag, Vmag
    FROM data
    JOIN photometry ON data.HIP = photometry.HIP
    WHERE RV > 0
    ORDER BY Bmag
    LIMIT 20;
    """

df2 = pd.read_sql_query(sql_command, con)
df2

,HIP,RV,Bmag,Vmag
0,30438,20.30,-0.456,-0.62
1,53,11.42,0.000,10.96
2,3937,6.00,0.000,12.01
3,4725,57.00,0.000,11.80
4,6115,130.00,0.000,11.63
5,8939,24.00,0.000,12.37
6,11517,9.00,0.000,11.85
7,12886,28.00,0.000,10.80
8,14574,11.80,0.000,11.72
9,14699,12.79,0.000,10.63


**Exercise 8:**

1. Using `sqlite3` and python, query the database for `B-V` and Absolute V mag (i.e. `VAbsMag`) measurements.
  - Select only sources that are have a Distance error (`e_Dist`) that is non-zero and less than 10. 
  - Also filter out non-zero `B-V` values
  - *Hint: You will need to join the `data` and `photometry` tables.*
  - *Hint: You will need to wrap 'B-V' in double-quotes in the `SELECT` statement to avoid errors caused by the minus-sign.*
2. Parse the results by creating a list that holds the `B-V` values and a list that holds the `VabsMag` values.
3. Plot the results using `pyplot` *(hint: It may be useful to invery the y-axis using `plt.ylim(plt.ylim()[::-1])`*

**(Bonus) Exercise 9:**

1. Execute the previous query, only this time add conditions to select only white dwarf stars (`"B-V" < 0.7`, `VAbsMag > 10`).
2. Overplot the white dwarf stars on the plot from Excercise 8 using a different marker color.